<a href="https://colab.research.google.com/github/sau-coder/NLP/blob/master/paragraph_to_takeaways.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)
    print(squad_dict)

    source_text = []
    target_text = []
    for i in squad_dict:
      source_text.append(i["source_text"])
    for target in squad_dict:
      target_text.append(target['target_text'])

    return source_text , target_text

source_text , target_text = read_squad('/content/drive/MyDrive/data.json')

In [ ]:
import json
with open('/content/drive/MyDrive/data.json', 'rb') as f:
        squad_dict = json.load(f)

source_text = []
target_text = []
for i in squad_dict:
        print(i)

In [ ]:
pip install transformers

In [ ]:
pip install sentencepiece

In [ ]:
from transformers import AutoTokenizer , MT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('bigscience/mt0-small')
model = MT5Model.from_pretrained('bigscience/mt0-small')

In [ ]:
from sklearn.model_selection import train_test_split

x_train , x_val , y_train , y_val = train_test_split(source_text , target_text , test_size = 0.2)

x_train_tokenized = tokenizer(x_train , padding = True , truncation = True , max_length = 512 , return_tensors="pt").input_ids
x_val_tokenized = tokenizer(x_val , padding = True , truncation = True , max_length = 512 )
y_train_tokenized = tokenizer(y_train , padding = True , truncation = True , max_length = 512 )
y_val_tokenized = tokenizer(y_val , padding = True , truncation = True , max_length = 512 )



In [ ]:
len(x_train),len(x_val)

In [ ]:
import torch

class SMDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.inputs["input_ids"][idx]).squeeze()
        target_ids = torch.tensor(self.targets["input_ids"][idx]).squeeze()
        return {"input_ids": input_ids, "labels": target_ids}

    def __len__(self):
        return len(self.inputs)

In [ ]:
train_dataset = SMDataset(x_train_tokenized , y_train_tokenized)
val_dataset = SMDataset(x_val_tokenized , y_val_tokenized)

outputs = model(input_ids=train_dataset["input_ids"], decoder_input_ids=val_dataset["input_ids"])

In [ ]:
train_dataset[5]

In [ ]:
from transformers import TrainingArguments , Trainer

In [ ]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs = 1,
    per_device_train_batch_size = 8,
    
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
)

trainer.train()